In [3]:
# import importance file
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
import time
from bs4 import BeautifulSoup
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [4]:
path = 'C:/Users/Lenovo/Downloads/chromedriver/chromedriver.exe'
# Create a Service object
service = Service(executable_path=path)
# Initialize ChromeOptions
options = webdriver.ChromeOptions()

In [5]:
# Create the WebDriver instance using the Service object
driver = webdriver.Chrome(service=service, options=options)
driver.maximize_window()
# Navigate to a website (for testing)


In [ ]:
ipo_data = {
    "Code" : [], "Name":[], "Date":[], "Price":[], "Share":[],
    "Amount":[], "Supply":[], "Status":[]    }

for i in range (1,33):
    url = f"https://sahamidx.com/?view=Stock.IPO&path=Stock&page={i}"
    driver.get(url)
    table_elements = driver.find_elements(By.XPATH, 
                                          "/html/body/table[1]/tbody/tr/td[2]/table/tbody/tr[3]/td/table/tbody")
    # filter table based on class name
    rows = table_elements[0].find_elements(By.XPATH, ".//tr[td[@class='menu_link']]")
    for row in rows:
        try:
            columns = row.find_elements(By.TAG_NAME, "td")
            if len(columns) == 2: continue

            code = columns[1].text
            name = columns[2].text
            date = columns[3].text
            price,share = columns[4].text.split()
            amount,supply = columns[5].text.split()
            status = columns[6].text
            
            print(f"Code     :{code}")
            print(f"name     :{name}")
            print(f"date     :{date}")
            print(f"price    :{price}")
            print(f"share    :{share}")
            print(f"amount   :{amount}")
            print(f"supply   :{supply}")
            print(f"status   :{status}")
            print("----------------------")
            
            ipo_data["Code"]  .append(code)
            ipo_data["Name"]  .append(name)
            ipo_data["Date"]  .append(date)
            ipo_data["Price"] .append(price)
            ipo_data["Share"] .append(share)
            ipo_data["Amount"].append(amount)
            ipo_data["Supply"].append(supply)
            ipo_data["Status"].append(status)
            
        except:continue

        # if i == 10: break


# content

In [ ]:
import pandas as pd
ipo_df = pd.DataFrame(ipo_data)
ipo_df.tail()

In [ ]:
import yfinance as yf
from datetime import datetime
# Get today's date
today_date = datetime.today()
# change date format
ipo_df['Date'] = ipo_df['Date'].apply(lambda x: datetime.strptime(x, '%a, %d-%b-%Y').strftime('%Y-%m-%d'))
# remove ','
ipo_df["Price"] = ipo_df["Price"].apply(lambda x:int(x.replace(",","")))
codes = ipo_df["Code"]

In [ ]:
price_data = {
    f"Day {k} Gain": [] for k in range(1, 8)
}
price_data.update({
    f"Day {k} High": [] for k in range(1, 8)
})

for j in range(len(codes)) :
    # Define the ticker symbol
    ticker_symbol = f"{codes[j]}.JK"
    print(codes[j])

    # Get data for a specific period
    stock_data = yf.download(ticker_symbol, start= ipo_df['Date'][j], end=today_date)
    stock_data['Close'] = pd.to_numeric(stock_data['Close'], errors='coerce')
    stock_data['High'] = pd.to_numeric(stock_data['High'], errors='coerce')
    open_price = pd.to_numeric(ipo_df["Price"][j], errors='coerce')

    try:
        first_day_data = str(stock_data.index[0]).split(" ")[0]
    except:
        first_day_data=False

    first_day_actual = ipo_df["Date"][j]
    
    for k in range(1,8):
        if len(stock_data) >= k and first_day_actual == first_day_data :
            last_candle = stock_data.iloc[k-1]
            gain = (last_candle["Close"] - open_price)/open_price * 100
            high = (last_candle["High"] - open_price)/open_price * 100
            print(f"{k} Candle Gain: {gain:.2f}")
            print(f"{k} Candle high: {high:.2f}")

        else : gain,high = None,None
        
        price_data[f"Day {k} Gain"].append(gain)
        price_data[f"Day {k} High"].append(high)
    print("\n")

In [ ]:
price_df = pd.DataFrame(price_data)
price_df.head()

In [ ]:
from time import sleep
emiten_data = {
    "Warran":[], "Sector":[],"SectorID":[], 
    "UW":[],"BAE":[],
}

for code in codes :
    print(f"Code     :{code}")
    
    while True:
        try:
            driver.get(f"https://sahamidx.com/?view=Stock.Profile&path=Stock&stock_code={code}")
            table = driver.find_elements(By.XPATH,"/html/body/table[1]/tbody/tr/td[2]/table/tbody/tr[3]/td/table/tbody")[0]
            break
        except:
            driver.refresh()
            sleep(2)

    rows = table.find_elements(By.TAG_NAME, "tr")[1:]

    try:
        uw = driver.find_element(By.XPATH,"//tr[td[text()='Penjamin Pelaksana Emisi Efek']]").text.split(":")[1]
    except:
        uw = ""
    
    try:
        sector = driver.find_element(By.XPATH,"//tr[td[text()='Sector']]").text.split(":")[1]
    except:
        sector = ""

    try:
        sectorid = driver.find_element(By.XPATH,"//tr[td[text()='Sector ID']]").text.split(":")[1]
    except:
        sectorid = ""
    
    try:
        bae = driver.find_element(By.XPATH,"//tr[td[text()='Biro Administrasi Efek']]").text.split(":")[1]
    except:
        bae = ""

    try:
        warran = driver.find_element(By.CSS_SELECTOR, "td[valign='top'][colspan='3']").text
    except:
        warran = 0

    print(f"warran   :{warran}")
    print(f"uw       :{uw}")
    print(f"bae      :{bae}")
    print(f"sector   :{sector}")
    print(f"sector id:{sectorid}")
    print(i)
    print("\n")

    emiten_data["Warran"].append(warran)
    emiten_data["UW"].append(uw)
    emiten_data["BAE"].append(bae)
    emiten_data["Sector"].append(sector)
    emiten_data["SectorID"].append(sectorid)


In [ ]:
emiten_df = pd.DataFrame(emiten_data)
emiten_df.head()

# driver.quit()